# Phase de Nettoyage des données

Les différentes données récupérées ont tout d'abord besoin d'être nettoyées pour les besoins du projet. Si les données fournies par l'EUDA sont déjà très propres et ne nécéssiteront qu'un léger travail de tri, voire éventuellement quelques merge utiles sur certaines tables, ce n'est pas le cas des données scrapées à partir de pages Wikipedia qui vont nous demander un travail de nettoyage plus approfondi.

## Nettoyage des données scrapées depuis Wikipedia

On commence par générer notre dataframe à partir du scraping des données de [l'article Wikipedia sur la légalisation du Cannabis par Pays](https://en.wikipedia.org/wiki/Legality_of_cannabis). Le code détaillé et commenté se trouve dans le fichier `scraping.py`, on se contente ici d'appeler nos fonctions.


In [1]:
from Data.scraping import scraping as s

url = "https://en.wikipedia.org/wiki/Legality_of_cannabis"

table = s.scrape_second_wikipedia_table(url)
data, header = s.table_to_list(table)
legal_pays = s.make_pandas(data, header)

legal_pays.head()

,Recreational,Medical,Notes
Afghanistan,Illegal,Illegal,Main article: Cannabis in Afghanistan Producti...
Albania,Illegal,Legal,Main article: Cannabis in Albania Prohibited b...
Algeria,Illegal,Illegal,Main article: Cannabis in Algeria
Andorra,Illegal,Illegal,Main article: Cannabis in Andorra
Angola,Illegal,Illegal,Main article: Cannabis in Angola


On remarque alors que notre Dataframe contient plusieurs éléments qui nécéssitent d'être nettoyés, modifiés, voire supprimés. Dans l'ordre on peut citer:

1. **Supprimer la dernier colonne concernant les "Notes"**, qui peuvent nous être utile à des fins d'interprétation, ou encore pour aider à la classification ou à l'interprétation plus tard, mais qui en l'état seront inutiles à la statistique descriptive ou à la construction de modèles plus tard.

2. **Eliminer de notre dataframe les pays non-européens**, puisque les données de l'EUDA ne s'intéresse qu'au cas européeen, et que c'est là le coeur de notre sujet.

3. Enfin, il nous faudra **regrouper les différentes états des variables sous un format clair** En l'état, on dispose pour chaque pays d'une description parfois unique et idividuelle de la législation dans le pays donné. L'idéal serait de ne garder qu'une variable binaire: Légal/Illégal. Cependant la réalité est plus complexe dans certains pays, et il nous faudra alors faire des choix, et les justifier, pour essayer de ne garder qu'un nombre limité d'état pour chaque variable.

### Suppression de la colonne des Notes

Cela n'est pas bien compliqué, on utilise simplement les outils de la librairie Pandas:

In [2]:
import pandas as pd

legal_pays_clean = legal_pays.drop(columns = "Notes")

legal_pays_clean.head()

,Recreational,Medical
Afghanistan,Illegal,Illegal
Albania,Illegal,Legal
Algeria,Illegal,Illegal
Andorra,Illegal,Illegal
Angola,Illegal,Illegal


### Tri pour ne garder que les pays Européens

Cette opération n'est elle aussi pas très compliquée, on va se servir d'une table de correspondance scrapée sur le [site national de Statistiques Candiennes](https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110) (ce afin d'avoir le nom des pays en anglais), qui nous sera d'ailleurs aussi utile plus tard afin de faire correspondre chaque pays de l'Union européenne avec son code ISO 3316.

In [3]:
from Data.scraping import scraping as s

url = "https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110"

table = s.scrape_first_table(url)
data, header = s.table_to_list(table)
table_corres = s.make_pandas(data)

table_corres.head()

,0,1
Belgium,BE,BEL
Bulgaria,BG,BGR
Croatia,HR,HRV
Cyprus,CY,CYP
Czechia,CZ,CZE


On doit néanmoins ajouter à la main certains pays non-membres de l'Union Européenne, mais présent dans les données que nous avons à disposition (souvent des pays candidats comme la Turquie)

In [4]:
import pandas as pd

table_corres.loc["Austria"] = ["AT","AUT"]
table_corres.loc["Türkiye"] = ["TR","TUR"]
table_corres.loc["Norway"] = ["NO","NOR"]

table_corres.tail()
print(table_corres)

              0    1
Belgium      BE  BEL
Bulgaria     BG  BGR
Croatia      HR  HRV
Cyprus       CY  CYP
Czechia      CZ  CZE
Denmark      DK  DNK
Estonia      EE  EST
Finland      FI  FIN
France       FR  FRA
Germany      DE  DEU
Greece       GR  GRC
Hungary      HU  HUN
Ireland      IE  IRL
Italy        IT  ITA
Latvia       LV  LVA
Lithuania    LT  LTU
Luxembourg   LU  LUX
Malta        MT  MLT
Netherlands  NL  NLD
Poland       PL  POL
Portugal     PT  PRT
Romania      RO  ROU
Slovakia     SK  SVK
Slovenia     SI  SVN
Spain        ES  ESP
Sweden       SE  SWE
Austria      AT  AUT
Türkiye      TR  TUR
Norway       NO  NOR


On effectue ensuite le tri des pays européens selon la manière suivante, en supprimant de la table principale toutes les lignes qui n'ont pas d'équivalent dans la table de correspondance.

In [5]:
# On renomme certains pays pour correspondre aux données de l'EUDA 
legal_pays_clean.rename(index={"Turkey":"Türkiye"}, inplace=True)
legal_pays_clean.rename(index={"Czech Republic":"Czechia"}, inplace=True)

for index in legal_pays_clean.index:
    if index in table_corres.index:
        pass
    else:
        legal_pays_clean = legal_pays_clean.drop(index = index)

legal_pays_clean.head()


,Recreational,Medical
Austria,Possession for personal use decriminalized as ...,"Dronabinol, Sativex, Nabilone[25]"
Belgium,Decriminalized up to 3 g (1⁄10 oz) or cultivat...,Sativex[31]
Bulgaria,Illegal,Illegal
Croatia,Decriminalized[71],Legal[72]
Cyprus,Illegal,Legal[73]


### Recodage de chaque élément dans une variable ternaire

Il nous faut désormais faire un travail de recodage des informations contenues dans le tableau, afin de ne garder au final qu'une variable avec trois états possibles:

- `Legal` si la consommation, la possession et la vente sont légales et encadrées dans le pays
- `Illegal` si la consommation, la possession et la vente sont illégale et passibles de poursuites judiciaires
- `Decriminalized` si la consommation et la possession sont décriminalisées, mais la vente est illégale. On ne fait pas ici de différence entre la pénalisation ou non du cannabis (la seule différence étant que dans le cas ou ce dernier est pénalisé, la consommation et la possession sont toujours passibles d'une amende, mais pas de poursuites légales)

Dans certains cas la classification s'avère très simples, dans d'autres il nous faudra bien justifier tel ou tel classement en fonction de la situation du pays. Nous n'avons donc pas d'autre choix que d'effectuer ce travail manuellement, au cas pas cas.

Nous détaillerons et justifions brièvement ci-dessous les choix de codage effectués. Pour cela nous nous sommes principalement appuyés sur [l'article Wikipedia](https://en.wikipedia.org/wiki/Legality_of_cannabis) dont est issu le tableau, ainsi que sur les articles détaillés concernant chaque pays, qu'on peut retrouver à partir du même tableau.

- Austria: Décriminalisé pour un usage récréatif, et légal pour usage médical. Même si seulement certains types de cannabinoïdes sont autorisés (Sativex notamment)

- Belgium: Même cas que pour l'Autriche. On compte l'autorisation du Sativex comme valant une légalisation dans le domaine médical. Pour ce qui est de la décriminalisation de l'usage récréatif, cette dernière est limités à un plant, mais on le compte tout de même comme dépénalisé.

**En règle générale, on comptera une substance comme Légalisée ou dépénalisée, même si il y a une certaine limite de poids ou de plants pour la possession ou la culture**

- Czechia: Décriminalisée pour l'usage récréatif, légal pour l'usage médical même si il existe des limites, mais on se tient à notre règle fixée précédemment

- Finland: Illégal pour usage récréatif mais une forte tolérance existe, et résulte en général seulement en une amende. On le code donc comme "Décriminalisé"

- France: Même cas que pour la Finlande, l'amende remplace la peine légale dans la plupart des cas. L'usage médical reste illégal.

- Georgia: Légal à la possession et à la consommation mais pas à la vente. On le comptera tout de même comme légal.

- Hungary: Seul le CBD est autorisé pour l'usage médical, on le compte donc comme illégal.


**Deuxième règle générale: la légalisation du CBD ne vaut pas légalisation du cannabis**


In [19]:
# Recodage à la main de toutes les informations nécéssaires.

for index in legal_pays_clean.index:
    content = legal_pays_clean.loc[index].values.flatten().tolist()
    content = content[:-1]
    for i in range(len(content)):
        if "Legal" in content[i]:
            content[i] = "Legal"
        elif "Illegal" in content[i]:
            content[i] = "Illegal"
        elif "Decriminalized" in content[i]:
            content[i] = "Decriminalized"

    legal_pays_clean.loc[index] = content

"""
legal_pays_clean.loc["Austria"] = ['Decriminalized','Legal']
legal_pays_clean.loc["Belgium"] = ['Decriminalized','Legal']
legal_pays_clean.loc["Croatia"] = ['Decriminalized','Legal']
legal_pays_clean.loc["Cyprus"] = ['Illegal','Legal']
legal_pays_clean.loc["Czechia"] = ['Decriminalized','Legal']
legal_pays_clean.loc["Czechia"] = ['Decriminalized','Legal']
"""



print(legal_pays_clean)

ValueError: Must have equal len keys and value when setting with an iterable